In [7]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv("/Users/marinagasco/Desktop/VS_Bootcamp/Módulo_3/mod-3-pareja-2/promoK-modulo3-sprint2-pair-programming-marina-ariana/world_data_full_apply_nulos.csv", index_col = 0)
df.head(2)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
0,Afghanistan,60.0,AF,58.1,652.230,323.0,32.49,93.0,Kabul,8.672,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,NaN,48.9,9.3,71.4,11.12,NaN,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28.748,9.0,11.78,355.0,Tirana,4.536,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,NaN,55.7,18.6,36.6,12.33,NaN,41.153332,20.168331,Europe


In [6]:
df.columns

Index(['country', 'density', 'abbreviation', 'agricultural_land', 'land_area',
       'armed_forces_size', 'birth_rate', 'calling_code', 'capital/major_city',
       'co2-emissions', 'cpi', 'cpi_change', 'currency-code', 'fertility_rate',
       'forested_area', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment', 'infant_mortality',
       'largest_city', 'life_expectancy', 'maternal_mortality_ratio',
       'minimum_wage', 'official_language', 'out_of_pocket_health_expenditure',
       'physicians_per_thousand', 'population',
       'population_labor_force_participation', 'tax_revenue', 'total_tax_rate',
       'unemployment_rate', 'urban_population', 'latitude', 'longitude',
       'continent'],
      dtype='object')

# Intervalo de Confianza para la Esperanza de Vida

    Contexto: Trabajas en una organización de salud global y deseas estimar el intervalo de confianza al 95% para la esperanza de vida promedio de cada continente.

    Nota: Cuidado porque puede que os salgan algunos nulos, genera un código para que no salgan nulos en los intervalos de confianza.

In [8]:
# lo primero que tenemos que hacer es calcular la estimación puntual, en este caso calcularemos la media de la duración de las llamadas
media_esperanza_vida = df["life_expectancy"].mean()

# después vamos a calcular el error estándar utilizando el método "sem" de la librería stats
error_duracion = stats.sem(df["life_expectancy"])

# definimos el nivel de confianza (95% en este caso)
nivel_confianza_duracion = 0.95

# calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1. 
grados_libertad_duracion = len(df["life_expectancy"]) - 1 #tamaño de la muestra -1. #Cantidad de valores que pueden variar dentro de un cálculo teniendo en cuenta la info que ya conocemos. 

# calculamos el valor crítico de la muestra
valor_critico_duracion = stats.t.ppf((1 + nivel_confianza_duracion) / 2, df=grados_libertad_duracion)

# calculamos el intervalo de confianza
limite_inferior_duracion = media_esperanza_vida - valor_critico_duracion * error_duracion
limite_superior_duracion = media_esperanza_vida + valor_critico_duracion * error_duracion

print("Intervalo de Confianza para la Duración de Llamadas Telefónicas:")
print(f"Media Muestral: {np.round(media_esperanza_vida, 2)}")
print(f"Error Estándar: {np.round(error_duracion, 2)}")
print(f"Nivel de Confianza: {nivel_confianza_duracion}")
print(f"Valor Crítico: {np.round(valor_critico_duracion, 2)}")
print(f"Intervalo de Confianza: ({np.round(limite_inferior_duracion, 2)}, {np.round(limite_superior_duracion, 2)})")

Intervalo de Confianza para la Duración de Llamadas Telefónicas:
Media Muestral: 72.28
Error Estándar: nan
Nivel de Confianza: 0.95
Valor Crítico: 1.97
Intervalo de Confianza: (nan, nan)


In [9]:
#lo hacemos con una función: 


def intervalo_confianza(dataframe, columna, nivel_confianza = float):
    """
    Calcula el intervalo de confianza para una columna específica en un DataFrame.
    Parámetros:
    -----------
    dataframe : DataFrame
        El DataFrame que contiene los datos de la muestra.
    columna : str
        El nombre de la columna para la cual deseas calcular el intervalo de confianza.
    nivel_confianza : float, opcional
        El nivel de confianza deseado para el intervalo de confianza (valor predeterminado es 0.95).
    Salida:
    -------
    None
        La función imprime en la consola la siguiente información:
        - La media muestral de la columna especificada.
        - El error estándar de la muestra.
        - El nivel de confianza utilizado.
        - El valor crítico calculado a partir de la distribución t de Student.
        - El intervalo de confianza calculado, que incluye el límite inferior y el límite superior.
    """
    # Calcular la media y el error estándar de la muestra
    media = dataframe[columna].mean()
    error = stats.sem(dataframe[columna])
    # calculamos los grados de libertad de la muestra. Recordad que debemos restar el total de datos que tenemos -1.
    grados_libertad = len(df[columna]) - 1
    # Calcular el valor crítico (utilizando la distribución t de Student)
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
    # Calcular el intervalo de confianza
    limite_inferior = media - valor_critico * error
    limite_superior = media + valor_critico * error
    print(f"Intervalo de Confianza para {columna}:")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Error Estándar: {np.round(error, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Valor Crítico: {np.round(valor_critico, 2)}")
    print(f"Intervalo de Confianza: ({np.round(limite_inferior, 2)}, {np.round(limite_superior, 2)})")

In [15]:
intervalo_confianza(df,["life_expectancy"],0.95)

Intervalo de Confianza para ['life_expectancy']:
Media Muestral: life_expectancy    72.28
dtype: float64
Error Estándar: [nan]
Nivel de Confianza: 0.95
Valor Crítico: 1.97
Intervalo de Confianza: (life_expectancy   NaN
dtype: float64, life_expectancy   NaN
dtype: float64)


In [17]:
df["life_expectancy"].isnull().sum() #debemos gestionar estos (8) nulos para poder calcular el intervalo de confianza.  

np.int64(8)

In [25]:
datos_sin_nan = df.dropna()

In [27]:
datos_sin_nan.isnull().sum()

country                                 0
density                                 0
abbreviation                            0
agricultural_land                       0
land_area                               0
armed_forces_size                       0
birth_rate                              0
calling_code                            0
capital/major_city                      0
co2-emissions                           0
cpi                                     0
cpi_change                              0
currency-code                           0
fertility_rate                          0
forested_area                           0
gasoline_price                          0
gdp                                     0
gross_primary_education_enrollment      0
gross_tertiary_education_enrollment     0
infant_mortality                        0
largest_city                            0
life_expectancy                         0
maternal_mortality_ratio                0
minimum_wage                      

In [30]:
datos_sin_nan.head(1)

,country,density,abbreviation,agricultural_land,land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change,currency-code,fertility_rate,forested_area,gasoline_price,gdp,gross_primary_education_enrollment,gross_tertiary_education_enrollment,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation,tax_revenue,total_tax_rate,unemployment_rate,urban_population,latitude,longitude,continent
14,Barbados,668.0,BB,23.3,430.0,1.0,10.65,1.0,Bridgetown,1.276,134.09,4.1,BBD,1.62,14.7,1.81,5.209000e+09,99.4,65.4,11.3,Bridgetown,79.1,27.0,3.13,English,45.2,2.48,287.025,65.2,27.5,35.6,10.33,89.431,13.193887,-59.543198,Central America


In [31]:
def intervalo_confianza(dataframe, columna, nivel_confianza = float):
    
    media = dataframe[columna].mean()
    error = stats.sem(dataframe[columna])
    grados_libertad = len(df[columna]) - 1
    valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, df=grados_libertad)
    limite_inferior = media - valor_critico * error
    limite_superior = media + valor_critico * error
    
    print(f"Intervalo de Confianza para {columna}:")
    print(f"Media Muestral: {np.round(media, 2)}")
    print(f"Error Estándar: {np.round(error, 2)}")
    print(f"Nivel de Confianza: {nivel_confianza}")
    print(f"Valor Crítico: {np.round(valor_critico, 2)}")
    print(f"Intervalo de Confianza: ({np.round(limite_inferior, 2)}, {np.round(limite_superior, 2)})")

In [32]:
intervalo_confianza(datos_sin_nan,["life_expectancy"],0.95)


#La media de la muestra (76,35 años) está dentro del rango del intervalo de confianza (que va de 71,6 - 81,76.)
#H0 --> La esperanza de vida SÍ es igual a la media: 76,35. 
#H1. --> La esperanza de vida NO es igual a la media: 76,35. 

Intervalo de Confianza para ['life_expectancy']:
Media Muestral: life_expectancy    76.35
dtype: float64
Error Estándar: [2.41]
Nivel de Confianza: 0.95
Valor Crítico: 1.97
Intervalo de Confianza: (life_expectancy    71.6
dtype: float64, life_expectancy    81.1
dtype: float64)


In [40]:
#vamos a hacer la misma función haciendo groupby. 

agrupación = datos_sin_nan.groupby("continent")["life_expectancy"].count()

agrupación

continent
Africa             1
Central America    2
Europe             2
Oceania            1
Name: life_expectancy, dtype: int64

In [43]:
agrupación = datos_sin_nan.groupby("continent")["life_expectancy"].agg( mean="mean", std="std", n="count" ).reset_index()

agrupación

,continent,mean,std,n
0,Africa,72.8,NaN,1
1,Central America,76.8,3.252691,2
2,Europe,82.2,0.141421,2
3,Oceania,67.3,NaN,1


In [ ]:
agrupación = datos_sin_nan.groupby("continent")["life_expectancy"].apply(intervalo_confianza)

# Intervalo de Confianza para el PIB

    Contexto: Eres una economista y quieres calcular el intervalo de confianza al 90% para el Producto Interno Bruto (PIB) de los continentes.